In [18]:
import pandas as pd 
import numpy as np
from scipy.sparse import hstack
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,chi2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path= '/content/gdrive/MyDrive/Data-Set_Assig_1/Data-Set_Assig_1/FNC_Bin_Train.csv'
fnc_train_data=pd.read_csv(path)
fnc_test_data=pd.read_csv('/content/gdrive/MyDrive/Data-Set_Assig_1/Data-Set_Assig_1/FNC_Bin_Test.csv')


In [4]:
df=pd.concat([fnc_train_data,fnc_test_data])

In [5]:
stance=df.Stance
df.drop(labels='Stance',axis=1,inplace=False)

,Body ID,articleBody,Headline
0,2215,"Recently, a second-grader from Kohl Isilkulsky...",Schoolboy almost killed by electric shock clai...
1,298,Although Canadian police have not released the...,Former Porn Star Behind YouTube Sensation ‘DC ...
2,2371,Breaking news: The Teletubbies’ solar infant d...,Say 'eh-oh!' to the Teletubbies SUN BABY - can...
3,1786,A British man who went to fight against Islami...,Video Purports To Show Beheading Of U.S. Journ...
4,731,Thousands of social media users are distributi...,Report: ISIS Leader Abu Bakr Al-Baghdadi Assas...
...,...,...,...
25408,2019,Alaska Airlines jetliner had to turn back to t...,Scorpion Sting Delays Oregon State Flight
25409,2131,"No, that is not a joke headline stolen from Th...",Argentina’s President Adopted A Jewish Godson ...
25410,1146,An Egyptian man has reportedly delivered a fin...,ISIS Treasurer Reportedly Steals $1M From Grou...
25411,668,Before Apple unveiled the Apple Watch last Sep...,Third boob likely a hoax


In [6]:
# count_vect=CountVectorizer(ngram_range=(1,2))
# articleBodyVect=count_vect.fit_transform(fnc_train_data['articleBody'].values)

In [7]:
# articleBodyVect.get_shape()

In [8]:
# count_vect=CountVectorizer(ngram_range=(1,2))
# HeadlineBodyVect=count_vect.fit_transform(fnc_train_data['Headline'].values)
# HeadlineBodyVect.get_shape()


In [9]:
tf_idf=TfidfVectorizer(min_df=5,ngram_range=(1,2))
art_tfidf_Vect=tf_idf.fit_transform(df['articleBody'].values)
art_tfidf_Vect.get_shape()

(70387, 304313)

In [10]:
tf_idf=TfidfVectorizer(min_df=5,ngram_range=(1,2))
Hl_tfidf_Vect=tf_idf.fit_transform(df['Headline'].values)
Hl_tfidf_Vect.get_shape()
#print(Hl_tfidf_Vect[1,:].toarray()[0])  

(70387, 19811)

In [11]:
# features=tf_idf.get_feature_names()
# len(features)

In [12]:
df_data = hstack((art_tfidf_Vect,Hl_tfidf_Vect)).tocsr()
df_data.get_shape()

(70387, 324124)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df_data, stance, test_size=0.20, random_state=1)

In [14]:
print(X_train.shape)
print(X_test.shape)

(56309, 324124)
(14078, 324124)


Feature Selection

In [15]:
X_train_best=SelectKBest(chi2,k=1000).fit_transform(X_train, y_train)
X_test_best=SelectKBest(chi2,k=1000).fit_transform(X_test, y_test)
X_train_best_pca=X_train_best.toarray()
X_test_best_pca=X_test_best.toarray()

PCA

In [16]:
from sklearn.decomposition import PCA

In [20]:
pca=PCA()
start_time=time.time()
pca_train=pd.DataFrame(pca.fit_transform(X_train_best_pca))
print('Time taken to fit the PCA: ',time.time()-start_time)

Time taken to fit the PCA:  17.542535305023193


In [21]:
pca=PCA()
start_time=time.time()
pca_test=pd.DataFrame(pca.fit_transform(X_test_best_pca))
print('Time taken to fit the PCA: ',time.time()-start_time)

Time taken to fit the PCA:  5.863933324813843


Random Forest Calssifier

In [22]:
from sklearn.ensemble import RandomForestClassifier

Without Feature Selection

In [23]:
rf=RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
present_time=time.time()
rf.fit(X_train,y_train)
print(time.time()-present_time)
y_pred1 = rf.predict(X_test)


3.362290382385254


In [24]:
print(accuracy_score(y_test,y_pred1))

0.7270208836482455


In [25]:
# confusion metric
conf=confusion_matrix(y_test,y_pred1)
print(conf)
f1score=f1_score(y_test, y_pred1)


[[    0  3843]
 [    0 10235]]


With Feature Selection

In [26]:
rf2=RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
present_time=time.time()
rf2.fit(X_train_best,y_train)
print(time.time()-present_time)
y_pred2 = rf2.predict(X_test_best)


0.9073235988616943


In [27]:
print(accuracy_score(y_test,y_pred2))

0.7270208836482455


In [28]:
# confusion metric
conf=confusion_matrix(y_test,y_pred2)
print(conf)
f1score=f1_score(y_test, y_pred2)
print(f1score)

[[    0  3843]
 [    0 10235]]
0.8419364126187636


Using PCA

In [29]:
rf_pca=RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
present_time=time.time()
rf_pca.fit(pca_train,y_train)
print(time.time()-present_time)
y_pred_pca = rf_pca.predict(pca_test)

12.414392232894897


In [30]:
print(accuracy_score(y_test,y_pred_pca))

0.7270208836482455


In [31]:
# confusion metric
conf=confusion_matrix(y_test,y_pred_pca)
print(conf)
f1score=f1_score(y_test, y_pred_pca)
print(f1score)

[[    0  3843]
 [    0 10235]]
0.8419364126187636


Decison Tree

In [32]:
from sklearn.tree import DecisionTreeClassifier

Without feature Selection

In [33]:
dt=DecisionTreeClassifier(random_state=0)
start_time=time.time()
dt.fit(X_train, y_train)

print(time.time()-start_time)
y_pred = dt.predict(X_test)

312.40538573265076


In [34]:
print(accuracy_score(y_test,y_pred))

0.9519107827816451


In [35]:
# confusion metric
conf=confusion_matrix(y_test,y_pred)
print(conf)
f1score=f1_score(y_test, y_pred)
print(f1score)


[[3440  403]
 [ 274 9961]]
0.9671343269090731


With Feature Selection

In [36]:
dt=DecisionTreeClassifier(random_state=0)
start_time=time.time()
dt.fit(X_train_best, y_train)
print(time.time()-start_time)
y_pred = dt.predict(X_test_best)

3.2684435844421387


In [37]:
print(accuracy_score(y_test,y_pred))

0.6547805085949708


In [38]:
# confusion metric
conf=confusion_matrix(y_test,y_pred)
print(conf)
f1score=f1_score(y_test, y_pred)
print(f1score)

[[1559 2284]
 [2576 7659]]
0.75914362176628


Using PCA

In [39]:
dt=DecisionTreeClassifier(random_state=0)
start_time=time.time()
dt.fit(pca_train, y_train)

print(time.time()-start_time)
y_pred_pca = dt.predict(pca_test)

51.55425190925598


In [40]:
print(accuracy_score(y_test,y_pred_pca))

0.6095326040630772


In [41]:
conf=confusion_matrix(y_test,y_pred_pca)
print(conf)
f1score=f1_score(y_test, y_pred_pca)
print(f1score)

[[1798 2045]
 [3452 6783]]
0.7116403504170383


Naive Bayes Classifier

In [42]:
from sklearn.naive_bayes import MultinomialNB

In [43]:
Nb=MultinomialNB()
start_time=time.time()
Nb.fit(X_train, y_train)
print("time:",time.time()-start_time)
y_pred = Nb.predict(X_test)
print('Accuracy_score: ',accuracy_score(y_test,y_pred))

time: 0.30837464332580566
Accuracy_score:  0.807714163943742


In [44]:
# confusion metric
conf=confusion_matrix(y_test,y_pred)
print(conf)
f1score=f1_score(y_test, y_pred)


[[1895 1948]
 [ 759 9476]]


With Feature Selection

In [45]:
Nb=MultinomialNB()
start_time=time.time()
Nb.fit(X_train_best, y_train)
print("time:",time.time()-start_time)
y_pred = Nb.predict(X_test_best)
print('Accuracy_score: ',accuracy_score(y_test,y_pred))

time: 0.017049312591552734
Accuracy_score:  0.7358999857934365


In [46]:
# confusion metric
conf=confusion_matrix(y_test,y_pred)
print(conf)
f1score=f1_score(y_test, y_pred)


[[ 992 2851]
 [ 867 9368]]


Using PCA

In [47]:
GaussNb=GaussianNB()
start_time=time.time()
GaussNb.fit(pca_train, y_train)
print("time:",time.time()-start_time)
y_pred =GaussNb.predict(pca_test)
print('Accuracy_score: ',accuracy_score(y_test,y_pred))

time: 0.8644392490386963
Accuracy_score:  0.7175024861486007


In [48]:
# confusion metric
conf=confusion_matrix(y_test,y_pred)
print(conf)
f1score=f1_score(y_test, y_pred)
print(f1score)

[[1732 2111]
 [1866 8369]]
0.8080135167752837


# SVM

In [49]:
from sklearn import svm

Withouth Feature Selection 

In [ ]:
clf=svm.SVC()
start_time=time.time()
clf.fit(X_train,y_train)
print(time.time()-start_time)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))

In [ ]:
# confusion metric
conf=confusion_matrix(y_test,y_pred)
print(conf)
f1score=f1_score(y_test,y_pred)

With Feature selection

In [ ]:
clf=svm.SVC()
start_time=time.time()
clf.fit(X_train_best,y_train)
print(time.time()-start_time)
y_pred = clf.predict(X_test_best)
print(accuracy_score(y_test,y_pred))

In [ ]:
# confusion metric
conf=confusion_matrix(y_test,y_pred)
print(conf)
f1score=f1_score(y_test, y_pred)

With Pca

In [ ]:
clf=svm.SVC()
start_time=time.time()
clf.fit(pca_train,y_train)
print(time.time()-start_time)
y_pred = clf.predict(pca_test)
print(accuracy_score(y_test,y_pred))

In [ ]:
# confusion metric
conf=confusion_matrix(y_test,y_pred)
print(conf)
f1score=f1_score(y_test, y_pred)